# EM简要回顾
## 作者：hschen0712

EM算法主要用于求解包含隐变量模型的最大化似然估计问题，EM框架中我们通常将$ln\ p(\mathbf{X}|\theta)$称为不完全对数似然（incomplete log data likelihood)，而$ln\ p(\mathbf{X},\mathbf{Z}|\theta)$称为完全对数似然(complete log data likelihood )，EM算法的目标是通过最大化完全对数似然找到参数的合理估计。然而由于我们对隐变量一无所知，因此没办法直接最大化完全对数似然，替代方案是最大化完全对数似然的后验数学期望，具体地该算法的参数迭代更新式如下：
$$\theta^{(t+1)} = \arg\max_{\theta} \int ln \ p(\mathbf{X},\mathbf{Z}|\theta)\cdot p(\mathbf{Z}|\mathbf{X},\theta^{(t)}) d\mathbf{Z} =\arg\max_{\theta}\mathbb{E}_{p(\mathbf{Z}|\mathbf{X},\theta^{(t)})}[ln\ p(\mathbf{X},\mathbf{Z}|\theta)]$$
此处以高斯混合模型（GMM）为例，设$\mathbf{X}=\{x_1,x_2,...,x_n\}$，$\mathbf{Z}=\{z_1,...,z_n\}$（$Z$不一定要与$X$一一对应）。可以证明，**最大化完全对数似然关于后验分布$p(\mathbf{Z}|\mathbf{X},\theta^{(t)})$的期望等价于最大化不完全对数似然**。具体证明过程如下，首先我们可以应用贝叶斯公式把不完全数据似然函数写成
$$p(\mathbf{X}|\theta)=\frac{p(\mathbf{X},\mathbf{Z}|\theta)}{p(\mathbf{Z}|\mathbf{X},\theta)}$$
两边取对数，有
$$ln\ p(\mathbf{X}|\theta)=ln\ p(\mathbf{X},\mathbf{Z}|\theta) - ln\ p(\mathbf{Z}|\mathbf{X},\theta)$$
两边关于分布$p(\mathbf{Z}|\mathbf{X},\theta^{(t)})$求数学期望，有
$$ ln\ p(\mathbf{X}|\theta)=\int ln\ p(\mathbf{X},\mathbf{Z}|\theta)p(\mathbf{Z}|\mathbf{X},\theta^{(t)})d\mathbf{Z}-\int ln\ p(\mathbf{Z}|\mathbf{X},\theta) p(\mathbf{Z}|\mathbf{X},\theta^{(t)})d\mathbf{Z}$$
我们要证明$ln\ p(\mathbf{X}|\theta^{(t+1)})\geq ln\ p(\mathbf{X}|\theta^{(t)})$（即经过一轮迭代不完全对数似然必定上升），因此作一个差值
$$ ln\ p(\mathbf{X}|\theta^{(t+1)})- ln\ p(\mathbf{X}|\theta^{(t)})=(\int ln\ p(\mathbf{X},\mathbf{Z}|\theta^{(t+1)})p(\mathbf{Z}|\mathbf{X},\theta^{(t)}) d\mathbf{Z}-\int ln\ p(\mathbf{X},\mathbf{Z}|\theta^{(t)})p(\mathbf{Z}|\mathbf{X},\theta^{(t)}) d\mathbf{Z})-\int ln\ \frac{p(\mathbf{Z}|\mathbf{X},\theta^{(t+1)})}{p(\mathbf{Z}|\mathbf{X},\theta^{(t)})}p(\mathbf{Z}|\mathbf{X},\theta^{(t)}) d\mathbf{Z}$$
注意到$-\int ln\ \frac{p(\mathbf{Z}|\mathbf{X},\theta^{(t+1)})}{p(\mathbf{Z}|\mathbf{X},\theta^{(t)})}p(\mathbf{Z}|\mathbf{X},\theta^{(t)}) d\mathbf{Z}$实际上是KL散度，我们可以用琴生不等式证明它一定是非负的。那么
$$ln\ p(\mathbf{X}|\theta^{(t+1)})- ln\ p(\mathbf{X}|\theta^{(t)})\geq \int ln\ p(\mathbf{X},\mathbf{Z}|\theta^{(t+1)})p(\mathbf{Z}|\mathbf{X},\theta^{(t)}) d\mathbf{Z}-\int ln\ p(\mathbf{X},\mathbf{Z}|\theta^{(t)})p(\mathbf{Z}|\mathbf{X},\theta^{(t)}) d\mathbf{Z}$$
当$\theta^{(t+1)} = \arg\max_{\theta} \int ln \ p(\mathbf{X},\mathbf{Z}|\theta)\cdot p(\mathbf{Z}|\mathbf{X},\theta^{(t)}) d\mathbf{Z}$时这个差值就会大于等于0，于是每一轮迭代都会导致不完全对数似然增加，直到收敛。
概括一下EM算法的步骤：
首先我们给参数设定一个初值$\theta^{(0)}$，接着在以下两部间循环直至目标函数收敛
1）expectation：计算后验分布$p(\mathbf{Z}|\mathbf{X},\theta^{(t)})$和期望$\mathbb{E}_{p(\mathbf{Z}|\mathbf{X},\theta^{(t)})}[ln\ p(\mathbf{X},\mathbf{Z}|\theta)]$，这个期望是个关于$\theta$的函数
2）maximization：令$f(\theta)=\mathbb{E}_{p(\mathbf{Z}|\mathbf{X},\theta^{(t)})}[ln\ p(\mathbf{X},\mathbf{Z}|\theta)]$，寻找使$f(\theta)$最大的$\theta^{(t+1)}$
下面我们结合GMM叙述EM算法的主要步骤，首先我们假设$(x_i,z_i)$之间相互独立，则完全数据似然的计算方法为
$$p(\mathbf{X},\mathbf{Z}|\theta)=\prod_{i=1}^n p(x_i,z_i|\theta)=\prod_{i=1}^n p(x_i|z_i,\theta)p(z_i|\theta)=\prod_{i=1}^n \alpha_{z_i} \mathcal{N}(x_i|\mu_{z_i},\Sigma_{z_i})$$
对数完全数据似然
$$ ln\  p(\mathbf{X},\mathbf{Z}|\theta)=\sum_{i=1}^n ln\ (\alpha_{z_i} \mathcal{N}(x_i|\mu_{z_i},\Sigma_{z_i}))$$
同时我们假设$z_i|x_i$也相互独立，于是$p(\mathbf{Z}|\mathbf{X},\theta^{(t)})$可以写成
$$ p(\mathbf{Z}|\mathbf{X},\theta^{(t)})=\prod_{i=1}^n p(z_i|x_i,\theta^{(t)})=\prod_{i=1}^n\frac{p(x_i|z_i,\theta^{(t)})p(z_i|\theta^{(t)})}{\sum_{z_i=1}^K p(x_i|z_i,\theta^{(t)})p(z_i|\theta^{(t)})}= \prod_{i=1}^n\frac{\alpha^{(t)}_{z_i} \mathcal{N}(x_i|\mu^{(t)}_{z_i},\Sigma^{(t)}_{z_i})}{\sum_{l=1}^K \alpha^{(t)}_{l} \mathcal{N}(x_i|\mu^{(t)}_{l},\Sigma^{(t)}_{l})}$$


# EM和VI区别以及联系
## 区别
EM认为完全似然关于后验概率的期望是可以计算的，并通过期望最大化更新这个参数，EM算法是得到的估计是点估计；VI认为后验不可计算，因此需要估计，VI的目的是求得一个隐变量的分布，因此是区间估计。
EM假设分布的形式已知（可以将其参数化），从而能够将参数与隐变量进行分离，通过最大似然法估计参数；而VI则认为参数也是隐变量的一部分，于是我们不知道分布的样子，只能通过简单的分布估计推得。
## 联系
两者都将分布估计视为优化问题，两者的优化过程都使用到了坐标上升的思想。
# Variational EM
待续...
